In [1]:
import pandas as pd
import numpy as np
import os
import subprocess

In [2]:
main_model_dir="/home/hrenduchinta/LM-pretraining/models/gpt2_random_fixed_13_01_2023_08.07.20"

In [3]:
for step in range(50000, 550000, 50000):
    model_path=f"{main_model_dir}/step_{step}"
    try:
        data=pd.DataFrame(columns=["cola", "mrpc", "rte", "stsb", "sst2", "mnli-m", "mnli-mm", "qnli", "qqp"], index=[f"glue_run_{i}" for i in range(1, 21)])
        for run in range(1, 21):
            if run>5:
                tasks=["cola", "mrpc", "rte", "stsb"]
            else:
                tasks=["cola", "mrpc", "rte", "stsb", "sst2", "mnli", "qnli", "qqp"]
            l=[]
            for task in tasks:
                with open(f"{model_path}/glue_run_{run}/{task}.log", "r") as f:
                    lines=f.readlines()
                    if task=="mnli":
                        l.append(100*float(lines[-2].split()[-1].rstrip().rstrip('}')))
                    l.append(100*float(lines[-1].split()[-1].rstrip().rstrip('}')))
            if run>5:
                l.extend([np.nan]*5)
            data.loc[f"glue_run_{run}"]=l
        data.loc["average"]=data.mean(axis=0, skipna=True)
        data.loc["stddev"]=data.std(axis=0, skipna=True)
        data.loc["decrement_avg"]=[np.nan for i in range(9)]
        data.loc["maximum"]=data.max(axis=0, skipna=True)
        data.loc["decrement_max"]=[np.nan for i in range(9)]
        data.to_csv(f"{model_path}/glue_results.csv")
    except Exception as e:
        print(f"step-{step},{run},{task}: {str(e)}")

In [4]:
for step in range(50000, 300000, 50000):
    for run in range(1, 21):
        if run>5:
            tasks=["cola", "mrpc", "rte", "stsb"]
        else:
            tasks=["cola", "mrpc", "rte", "stsb", "sst2", "mnli", "qnli", "qqp"]
        for task in tasks:
            if not os.path.isfile(f"{main_model_dir}/step_{step}/glue_run_{run}/{task}.log"):
                print(step, run, task)

In [5]:
model_dir=main_model_dir
os.makedirs(f"{model_dir}/results", exist_ok=True)
for step in range(50000, 550000, 50000):
    os.makedirs(f"{model_dir}/results/step_{step}", exist_ok=True)
    subprocess.run(f"cp {model_dir}/step_{step}/glue_results.csv {model_dir}/results/step_{step}/glue_results.csv", shell=True)